In [1]:
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasetraw = load_dataset("csv",data_files="data/climate_fever_votes.csv") 
datasetraw

DatasetDict({
    train: Dataset({
        features: ['pair', 'voter_1', 'voter_2', 'voter_3', 'voter_4', 'voter_5'],
        num_rows: 7675
    })
})

In [3]:
dataset = datasetraw.map(remove_columns=["voter_2","voter_3","voter_4","voter_5"])
dataset = dataset.rename_columns({'pair':'text', 'voter_1':'label'})
dataset = dataset["train"].train_test_split(test_size=0.2)
dataset.column_names

{'train': ['text', 'label'], 'test': ['text', 'label']}

In [4]:
model_name = 'distilbert-base-uncased'


tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:  33%|███▎      | 2000/6140 [00:00<00:01, 3556.52 examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer",num_train_epochs=1)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

  0%|          | 3/3840 [00:35<12:47:04, 11.99s/it]

In [ ]:
trainer.evalute()